# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

In [ ]:
import pandas as pd
import regex as re

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны.

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [ ]:
import pandas as pd
import random

recipes = pd.read_csv('recipes_sample.csv')
random_recipes = recipes.sample(5)

# Calculate the maximum width for each column
max_width_id = max(len(str(id)) for id in random_recipes['id'])
max_width_minutes = max(len(str(minutes)) for minutes in random_recipes['minutes'])

#создаем таблицу({}- задает ширину полученную )
table = "|{:^{}}|{:^{}}|\n".format('id', max_width_id, 'minutes', max_width_minutes)
table += "-" * (max_width_id + max_width_minutes + 4) + "\n"

for index, row in random_recipes.iterrows():
    table += "|{:^{}}|{:^{}}|\n".format(row['id'], max_width_id, row['minutes'], max_width_minutes)

print(table)

|  id  |minutes|
-------------
|277058|130|
|65112 |40 |
|100776| 5 |
|55875 |15 |
|138305|30 |



2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__).
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
import xml.etree.ElementTree as ET
import csv

def show_info(contributor_id, minutes, recipe_id):
    with open('recipes_sample.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        recipe_data = next((row for row in reader if row['id'] == recipe_id), None)

    if recipe_data is None:
        return "No recipe found with the given id."

    name = recipe_data['name']
    name = name.title()
    name = ' '.join(word for word in name.split() if word)

    steps_tree = ET.parse('steps_sample.xml')
    steps_root = steps_tree.getroot()
    steps_list = [step.text for step in steps_root.findall('.//step') if step.get('recipeId') == recipe_id]

    steps_str = '\n'.join([f'{index + 1}. {step}' for index, step in enumerate(steps_list)])
    divider = '-' * len(name)

    info_str = f"{name}\n\n{steps_str}\n\n----------\nАвтор: {contributor_id}\nСреднее время приготовления: {minutes} минут"
    return info_str

recipe_info = show_info( recipe_id=170895)
print(recipe_info)

TypeError: show_info() missing 2 required positional arguments: 'contributor_id' and 'minutes'

In [ ]:
import xml.etree.ElementTree as ET
recipes_data = pd.read_csv('recipes_sample.csv')
steps_data = ('steps_sample.xml')

def show_info(recipe_id, recipes_data, steps_data):
    recipe_info = recipes_data[recipes_data["id"] == recipe_id]
    steps_info = steps_data[steps_data["recipe_id"] == recipe_id]

    recipe_name = recipe_info["name"].values[0]
    contributor_id = recipe_info["contributor_id"].values[0]
    minutes = recipe_info["minutes"].values[0]

    steps_list = [f"{i+1}. {step}" for i, step in enumerate(steps_info["description"])]

    info_string = f'"{recipe_name}"\n\n' + '\n'.join(steps_list) + '\n----------\n' + f'Автор: {contributor_id}\n' + f'Среднее время приготовления: {minutes} минут'

    return info_string
recipe_id = 170895
info = show_info(recipe_id, recipes_data, steps_data)
print(info)  # Вывод полученной строки на экран


In [ ]:
import csv
import xml.etree.ElementTree as ET

def show_info(recipes_id):
    # Load recipes data from CSV file
    with open('recipes_sample.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        recipes = {row['id']: row for row in reader}

    # Load steps data from XML file
    tree = ET.parse('steps_sample.xml')
    root = tree.getroot()
    steps = {step.attrib['recipe_id']: [step.text for step in step.findall('step')] for step in root.findall('recipe')}

    # Get recipe and steps data
    recipe = recipes.get(recipe_id)
    steps_data = steps.get(recipe_id)

    if not recipe or not steps_data:
        return f"Recipe with id {recipe_id} not found."

    # Create recipe description string
    recipe_description = f"\n\"{recipe['title']}\"\n\n"
    for i, step in enumerate(steps_data, start=1):
        recipe_description += f"{i}. {step}\n"
    recipe_description += "----------\n"
    recipe_description += f"Автор: {recipe['contributor_id']}\n"
    recipe_description += f"Среднее время приготовления: {recipe['cooking_time']} минут\n"

    return recipe_description

# Call the function for recipe with id 170895 and print the result
print(show_info(170895))

In [ ]:
recipe_id = 170895
info = show_info(recipe_id, recipes_data, steps_data)
print(info)  # Вывод полученной строки на экран

In [ ]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [ ]:
tree = ET.parse("steps_sample.xml")
root = tree.getroot()

recipe_id = 25082
steps = []
for recipe in root.findall('recipe'):
    if recipe.find('id').text == str(recipe_id):
        steps = [step.text.strip() for step in recipe.find('steps').findall('step')]

# ?: - не захватывающая группа
pattern = r'\d+\s+(?:hour|hours|minute|minutes)'
for step in steps:
    matches = re.findall(pattern, step)
    if matches:
        print(matches)

['20 minute']
['10 minute']
['2 hour']
['10 minute']
['20 minute', '30 minute']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [ ]:
import pandas as pd
import re

recipes = pd.read_csv('recipes_sample.csv', header=0)
pattern = r'^this([\w\d\s]+),\s?but'
for desc in recipes['description']:
    if isinstance(desc, str):
        match = re.search(pattern, desc)
        if match:
            print(match.group())

this is a great meal eaten the same day ,but
this was adapted from a recipe i found on the net, but
this is kind of similar to some of the other versions out there, but
this is a moist, but
this pie does not have very many ingredients in it, but
this is very different, but
this is really to simple to call a recipe,but
this dressing is wonderful on baby greens salad, but
this is a great recipe, but
this casserole may be made with regular white rice, but
this salad has quite a few ingredients, but
this marinade is wonderful for any meat, but
this recipe shows for pork, but
this is considered peasant soup, but
this recipe is posted by request, but
this is a terribly sinful dish, but
this is not only fantastic hot, but
this is so gringo it is unreal, but
this recipe is so good you will want to save it for company, but
this cake is very rich, but
this soup is served as a fancy appetizer in a shot glass with half a piece of candied bacon, but
this is a rich creamy drink made of vodka, but
th

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [ ]:
import pandas as pd
import re
import xml.etree.ElementTree as ET

tree = ET.parse("steps_sample.xml")
root = tree.getroot()

recipe_id = 72367
steps = []
for recipe in root.findall('recipe'):
    if recipe.find('id').text == str(recipe_id):
        steps = [step.text.strip() for step in recipe.find('steps').findall('step')]

pattern = r'(\d+)\s?/\s?(\d+)'
i = -1
for step in steps:
    i += 1
    match = re.search(pattern, step)
    if match:
        new_s = step[:match.start() + 1] + match.group(1) + '/' + match.group(2) + step[match.end():]
        steps[i] = new_s

for step in steps:
    print(step)

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import xml.etree.ElementTree as ET

def tokenize_to_words(s):
    words = word_tokenize(s.lower())
    words = [word for word in words if word.isalpha()] # Только алфавитные слова
    return words


tree = ET.parse("steps_sample.xml")
root = tree.getroot()

all_words = []
for recipe in root.findall('recipe'):
    steps = [step.text.strip() for step in recipe.find('steps').findall('step')]
    for step in steps:
        all_words.extend(tokenize_to_words(step))

unique_words = set(all_words)

print("Количество уникальных слов:", len(unique_words))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\kiril/nltk_data'
    - 'C:\\Users\\kiril\\anaconda3\\nltk_data'
    - 'C:\\Users\\kiril\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\kiril\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\kiril\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [ ]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from nltk.tokenize import sent_tokenize
import xml.etree.ElementTree as ET
import pandas as pd

def tokenize_to_sentences(text):
    sentences = sent_tokenize(text)
    return sentences


recipes = pd.read_csv('recipes_sample.csv', header=0)

# Разбиение всех значений в столбце description на их предложения, преобразование в их количество и получение пяти записей с наибольшими значеняими
longest = recipes.loc[recipes['description'].apply(lambda x: isinstance(x, str))]['description'].apply(tokenize_to_sentences).apply(len).nlargest(5)
# print(longest)

for idx, num_sentences in longest.items():
    description = recipes.iloc[idx]['description']
    print(f"---------------------------------------------------------------------------"
          f"\nРецепт #{idx} ({num_sentences} предложений):\n{description}\n")

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\kiril/nltk_data'
    - 'C:\\Users\\kiril\\anaconda3\\nltk_data'
    - 'C:\\Users\\kiril\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\kiril\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\kiril\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
```
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def pos_tag_sentence(sentence):
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    return pos_tags

try:
    pos_tag_sentence('')
except LookupError:
    import nltk
    nltk.download('averaged_perceptron_tagger')

def process(pos_tags):
    output = ""
    for token, pos_tag in pos_tags:
        output += f"{pos_tag:<5}"
    output += "\n"

    for token, _ in pos_tags:
        output += f"{token} "

    return output


recipe_id = 241106
recipes = pd.read_csv('recipes_sample.csv', header=0)
recipe_data = recipes[recipes['id'] == recipe_id]

print(process(pos_tag_sentence(recipe_data['name'].values[0])))